In [3]:
import torch

ckpt = torch.load("/data/code/DQN-MedSAM2/checkpoints/sam2_hiera_tiny.pt", map_location="cpu")
for k in ckpt["model"].keys():
    print(k)

maskmem_tpos_enc
no_mem_embed
no_mem_pos_enc
no_obj_ptr
image_encoder.trunk.pos_embed
image_encoder.trunk.pos_embed_window
image_encoder.trunk.patch_embed.proj.weight
image_encoder.trunk.patch_embed.proj.bias
image_encoder.trunk.blocks.0.norm1.weight
image_encoder.trunk.blocks.0.norm1.bias
image_encoder.trunk.blocks.0.attn.qkv.weight
image_encoder.trunk.blocks.0.attn.qkv.bias
image_encoder.trunk.blocks.0.attn.proj.weight
image_encoder.trunk.blocks.0.attn.proj.bias
image_encoder.trunk.blocks.0.norm2.weight
image_encoder.trunk.blocks.0.norm2.bias
image_encoder.trunk.blocks.0.mlp.layers.0.weight
image_encoder.trunk.blocks.0.mlp.layers.0.bias
image_encoder.trunk.blocks.0.mlp.layers.1.weight
image_encoder.trunk.blocks.0.mlp.layers.1.bias
image_encoder.trunk.blocks.1.norm1.weight
image_encoder.trunk.blocks.1.norm1.bias
image_encoder.trunk.blocks.1.attn.qkv.weight
image_encoder.trunk.blocks.1.attn.qkv.bias
image_encoder.trunk.blocks.1.attn.proj.weight
image_encoder.trunk.blocks.1.attn.proj.bi

/tmp/ipykernel_1037832/2579334622.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("/data/code/DQN-MedSAM2/checkpoints/sam2_hiera_tiny.pt", map_location

In [ ]:
from glob import glob

for task in glob(f"/data/datasets/Combined_Dataset/MSD/*"):
    print(task, len(glob(f"{task}/imagesTr/*")))

In [ ]:
import nibabel as nib
import numpy as np
import psutil
import os
import torch.nn.functional as F
import torch

def normalization(image):
    image_min = np.min(image)
    image_max = np.max(image)
    image = ((image - image_min)/(image_max-image_min))*255
    return image

process = psutil.Process(os.getpid())
mem_info = process.memory_info()
print(f"Initial memory usage: {mem_info.rss / (1024 * 1024):.2f} MiB")

image_3d = nib.load("/data/datasets/Combined_Dataset/MSD/Task01_BrainTumour/imagesTr/BRATS_433.nii.gz")
data_seg_3d = nib.load("/data/datasets/Combined_Dataset/MSD/Task01_BrainTumour/labelsTr/BRATS_433.nii.gz")
image_3d = image_3d.dataobj
data_seg_3d = data_seg_3d.dataobj
# image_3d = image_3d.get_fdata(dtype=np.float32)
# data_seg_3d = data_seg_3d.get_fdata(dtype=np.float32)

if image_3d.ndim == 4:
    if image_3d.shape[-1] == 4:
        image_3d = image_3d[..., 2]
    elif image_3d.shape[-1] == 2:
        image_3d = image_3d[..., 0]
        
image_3d = np.asanyarray(image_3d)
data_seg_3d = np.asanyarray(data_seg_3d)

pos_slices = np.sum(data_seg_3d, axis=(0,1)) > 0
image_3d = image_3d[:, :, pos_slices]
data_seg_3d = data_seg_3d[:, :, pos_slices]

max_slices = 16
if image_3d.shape[-1] > max_slices and max_slices > 0:
    start_slice = np.random.choice(range(image_3d.shape[-1] - max_slices + 1))
    image_3d = image_3d[..., start_slice:start_slice+max_slices]
    data_seg_3d = data_seg_3d[..., start_slice:start_slice+max_slices]

image_3d = normalization(image_3d)
image_3d = torch.rot90(torch.tensor(image_3d)).permute(2, 0, 1).unsqueeze(0).unsqueeze(0)
data_seg_3d = torch.rot90(torch.tensor(data_seg_3d)).permute(2, 0, 1).unsqueeze(0).unsqueeze(0)
# image_3d = torch.tensor(image_3d).permute(2, 0, 1).unsqueeze(0).unsqueeze(0)
# data_seg_3d = torch.tensor(data_seg_3d).permute(2, 0, 1).unsqueeze(0).unsqueeze(0)

image_3d = F.interpolate(image_3d, size=(image_3d.shape[2], 1024, 1024), mode='trilinear', align_corners=False)
data_seg_3d = F.interpolate(data_seg_3d, size=(data_seg_3d.shape[2], 1024, 1024), mode='nearest')
image_3d = image_3d.squeeze(0).repeat(3, 1, 1, 1).permute(1, 0, 2, 3)
data_seg_3d = data_seg_3d.squeeze(0).squeeze(0)

process = psutil.Process(os.getpid())
mem_info = process.memory_info()
print(f"Final memory usage: {mem_info.rss / (1024 * 1024):.2f} MiB")

Initial memory usage: 431.23 MiB


ValueError: Cannot handle fancy indexing